<a href="https://colab.research.google.com/github/vandana-iyer/RecSys-Instacart/blob/master/InstacartRecsysAlgorithmsSVD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Authenticate and download datasets from Kaggle

!mkdir -p ~/.kaggle
!wget -O ~/.kaggle/kaggle.json https://www.dropbox.com/s/b6g4cawc664i4u9/kaggle.json?dl=0
!chmod 600 ~/.kaggle/kaggle.json
!ls ~/.kaggle

!pip install --upgrade kaggle

!kaggle competitions download -c instacart-market-basket-analysis
!unzip instacart-market-basket-analysis.zip && rm instacart-market-basket-analysis.zip

# unzip all csv files
!unzip \*.csv.zip
# remove unwanted files/directories
!rm -rf __MACOSX
# !rm *.zip

--2020-05-09 16:54:31--  https://www.dropbox.com/s/b6g4cawc664i4u9/kaggle.json?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.9.1, 2620:100:601f:1::a27d:901
Connecting to www.dropbox.com (www.dropbox.com)|162.125.9.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/b6g4cawc664i4u9/kaggle.json [following]
--2020-05-09 16:54:31--  https://www.dropbox.com/s/raw/b6g4cawc664i4u9/kaggle.json
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uce7de8aed8f3ad359842589a2c0.dl.dropboxusercontent.com/cd/0/inline/A3YHy3aJ3ySg1pwHdiKqjkCnFM8M203X-XB7Q8290DSbZTP_UtZdpj-gnbLRWsz6QNVWHLBKjfek7C6_SuD20csATKlZ5gWlocwlLKIEiM6aC1mm80zCWSRGEDQY4zpcZqM/file# [following]
--2020-05-09 16:54:32--  https://uce7de8aed8f3ad359842589a2c0.dl.dropboxusercontent.com/cd/0/inline/A3YHy3aJ3ySg1pwHdiKqjkCnFM8M203X-XB7Q8290DSbZTP_UtZdpj-gnbLRWsz6QNVWHLBKjfek7C6_SuD20csATKlZ5gWlocwlLKIEiM6aC1mm

In [3]:
!pip install surprise

     |████████████████████████████████| 6.5MB 2.4MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.0-cp36-cp36m-linux_x86_64.whl size=1673651 sha256=1df2191bfe70889527a982e0a4b5143c6ff0bd0bab09c0cf647a2fca4c2c0bea
  Stored in directory: /root/.cache/pip/wheels/cc/fa/8c/16c93fccce688ae1bde7d979ff102f7bee980d9cfeb8641bcf
Successfully built scikit-surprise


In [0]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
aisles = pd.read_csv('aisles.csv')
departments = pd.read_csv('departments.csv')
orders = pd.read_csv('orders.csv')
products = pd.read_csv('products.csv')
order_products = pd.read_csv('order_products__train.csv')
order_products_prior = pd.read_csv('order_products__prior.csv')

In [6]:
# Merge products, aisles and departments
products_dept_aisles_df = products.merge(aisles, on='aisle_id').merge(departments, on='department_id')
products_dept_aisles_df.head()

,product_id,product_name,aisle_id,department_id,aisle,department
0,1,Chocolate Sandwich Cookies,61,19,cookies cakes,snacks
1,78,Nutter Butter Cookie Bites Go-Pak,61,19,cookies cakes,snacks
2,102,Danish Butter Cookies,61,19,cookies cakes,snacks
3,172,Gluten Free All Natural Chocolate Chip Cookies,61,19,cookies cakes,snacks
4,285,Mini Nilla Wafers Munch Pack,61,19,cookies cakes,snacks


In [7]:
# Merge order and product_orders

print(order_products.shape)
print(order_products_prior.shape)
print(orders.shape)

user_product_orders = order_products.append(order_products_prior).merge(orders, on='order_id')
user_product_orders.shape

(1384617, 4)
(32434489, 4)
(3421083, 7)


(33819106, 10)

In [8]:
# Merge all tables into a single dataset

data = user_product_orders.merge(products_dept_aisles_df, on='product_id')
data.shape

(33819106, 15)

#### User Department embeddings

In [9]:
%%time
# Dept User embeddings - total products 

user_department = data.groupby(['user_id', 'department_id']).agg({'product_id': 'count'}).rename(columns={'product_id': 'total_products'})

# Alternatively, use pivot_table as shown below, but is slower by few milliseconds seconds as compared to above groupby + unstack()
# data.pivot_table(values='product_id', index='user_id', columns='department_id', aggfunc='count')

CPU times: user 5.5 s, sys: 2.31 s, total: 7.81 s
Wall time: 7.81 s


In [0]:
# np_array_user_department = data_user_department.unstack('department_id').values
# np_array_user_department = np.nan_to_num(np_array_user_department).astype(int)
# np_array_user_department

In [0]:
user_department = user_department.reset_index()

In [12]:
%%time

from surprise import SVD
from surprise import Dataset
from surprise import Reader
from surprise import accuracy
from surprise.model_selection import PredefinedKFold

reader = Reader()

CPU times: user 82.8 ms, sys: 10.3 ms, total: 93.1 ms
Wall time: 92.6 ms


In [0]:
user_department_surprise_data = Dataset.load_from_df(user_department, reader)

In [14]:
%%time

from surprise.model_selection import cross_validate

# algo = SVDpp()
# cross_validate(algo, user_department_surprise_data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

CPU times: user 11 µs, sys: 0 ns, total: 11 µs
Wall time: 14.1 µs


In [0]:
# Let us see why there is a difference of almost 30+ product purchases for each department
# Users that have not bought more than 2 products from a particular department and
# users that have purchased more than 500 products accounts to about 
# 18% of the user_dept dataset.
# let us filter them out

user_department_filtered = user_department[(user_department['total_products'] >= 2) & (user_department['total_products'] < 200)] # RMSE worsens with these conditions.
user_counts = user_department_filtered['user_id'].value_counts()
user_counts_greater_than_two = user_counts[user_counts > 2].index.tolist()
user_department_filtered = user_department_filtered[user_department_filtered['user_id'].isin(user_counts_greater_than_two)]

# Let us design a test/train split such that user_ids and department_ids are proportionally distributed. 

# print(user_department['total_products'].min())
# print(user_department['total_products'].max())

# Check for outliers in max purchased and filter them

user_department_surprise_data = Dataset.load_from_df(user_department_filtered, reader)

In [0]:
from sklearn.model_selection import train_test_split 

trainset_sklearn, testset_sklearn = train_test_split(user_department_filtered, test_size=0.25, stratify=user_department_filtered['user_id']) 

trainset_sklearn.to_csv('trainset.csv', index=False, header=None, sep='\t')
testset_sklearn.to_csv('testset.csv', index=False, header=None, sep='\t')

folds_files = [('trainset.csv', 'testset.csv')]

train_test_data = Dataset.load_from_folds(folds_files, reader=reader)
pkf = PredefinedKFold()
algo = SVD()

In [0]:
for trainset, testset in pkf.split(train_test_data):

    # train and test algorithm.
    algo.fit(trainset)
    predictions = algo.test(testset)

    # Compute and print Root Mean Squared Error
    print(accuracy.rmse(predictions, verbose=True))

In [42]:
for trainset, testset in pkf.split(train_test_data):
  algo = SVD()
  cross_validate(algo, Dataset.load_from_df(trainset_sklearn, reader), measures=['RMSE', 'MAE'], cv=5, verbose=True)

  predictions = algo.test(testset)
  print(accuracy.rmse(predictions, verbose=True))

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    26.0176 26.0350 26.2214 26.1157 26.0305 26.0840 0.0769  
MAE (testset)     12.2901 12.3251 12.3440 12.3177 12.2967 12.3147 0.0195  
Fit time          72.73   73.34   73.36   73.78   74.60   73.56   0.62    
Test time         2.75    2.74    3.50    2.80    2.78    2.91    0.30    
RMSE: 26.4501
26.450116268709447


In [0]:
# Weird observations: time taken is more with lesser/filtered data. 
# I don't think measuring user_dept ratings is a good one, because suddenly we see a surge of products bought.
# About 1% of users purchased more than 150 products at a time from a particular dept.
# is that an outlier?

# Take log for faster computation
# SEPARATE DATA INTO WEEKLY PURCHASES

# But hold on - this way of using data does not support the business goal.


# See if you can do a weighted average by considering "is_reordered" column
pd.set_option('display.max_rows', 50000)

In [38]:
len(user_department_filtered[user_department_filtered['total_products'] > 200])
# user_department_filtered[user_department_filtered['user_id'] == 5]

12657

In [36]:
len(user_department_filtered)

2275883

In [127]:
algo = SVD()

# Train the algorithm on the trainset, and predict ratings for the testset
train_result = algo.fit(trainset)
predictions = algo.test(testset.build_testset())

# Then compute RMSE
accuracy.rmse(predictions)

KeyboardInterrupt: ignored

In [22]:
departments

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol
5,6,international
6,7,beverages
7,8,pets
8,9,dry goods pasta
9,10,bulk


In [73]:
print(user_department['total_products'].min())
print(user_department['total_products'].max())

user_department_filtered.head()

1
1577


,user_id,department_id,total_products
0,1,4,5
1,1,7,15
2,1,13,1
3,1,14,4
4,1,16,17


In [0]:
trainset_sklearn, testset_sklearn = train_test_split(user_department_filtered, test_size=0.25, stratify=user_department_filtered['user_id']) 

In [52]:
trainset_sklearn['user_id'].value_counts()

,user_id,department_id,total_products
769057,69470,20,2
769050,69470,4,4


In [44]:
testset_sklearn['user_id'].value_counts()

182194    5
115912    5
39779     5
113552    5
100574    5
         ..
199124    1
120936    1
118889    1
22974     1
69470     1
Name: user_id, Length: 175026, dtype: int64

In [19]:
from surprise.model_selection import GridSearchCV

param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
              'reg_all': [0.4, 0.6]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)

gs.fit(user_department_surprise_data)

print(gs.best_score)
# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

14.077267613423425
{'n_epochs': 5, 'lr_all': 0.002, 'reg_all': 0.4}


User Aisle embeddings

In [20]:
%%time

# User Aisle embeddings

user_aisle = data.groupby(['user_id', 'aisle_id']).agg({'product_id': 'count'}).rename(columns={'product_id': 'total_products'})

CPU times: user 7.12 s, sys: 20 ms, total: 7.14 s
Wall time: 7.15 s


In [23]:
np_array_user_aisle = user_aisle.unstack('aisle_id').values
np_array_user_aisle = np.nan_to_num(np_array_user_aisle).astype(int) # Replace NaNs with 0
np_array_user_aisle
# user_id = user_aisle.index.get_level_values(level=0)

array([[ 0,  0,  0, ...,  0,  0,  0],
       [ 2,  0,  7, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       ...,
       [ 0,  0,  6, ...,  0,  0,  0],
       [ 0,  0, 35, ...,  0,  0,  0],
       [ 0,  0,  3, ...,  0,  0,  0]])

In [0]:
user_aisle = user_aisle.reset_index()
surprise_user_aisle_data = Dataset.load_from_df(user_aisle, reader)

In [33]:
%%time

from surprise.model_selection import cross_validate

algo = SVD()
cross_validate(algo, surprise_user_aisle_data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    12.2337 12.2483 12.1960 12.1728 12.1093 12.1920 0.0493  
MAE (testset)     5.1699  5.1764  5.1713  5.1745  5.1666  5.1717  0.0034  
Fit time          286.16  288.27  289.85  292.32  295.43  290.40  3.22    
Test time         18.16   18.00   18.58   18.68   20.68   18.82   0.96    
CPU times: user 27min 13s, sys: 3.22 s, total: 27min 16s
Wall time: 27min 16s


In [0]:
from surprise.model_selection import train_test_split

trainset, testset = train_test_split(surprise_data, test_size=.25)

algo = SVD()

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)

RMSE: 12.1579


12.157881121039713

In [0]:
from surprise.model_selection import GridSearchCV

param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
              'reg_all': [0.4, 0.6]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)

gs.fit(surprise_data)
7
# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

12.192035659486308
{'n_epochs': 5, 'lr_all': 0.002, 'reg_all': 0.4}


In [0]:
%%time

# User product embeddings

user_product = data.groupby(['user_id', 'product_id']).agg({'product_id': 'count'}).rename(columns={'product_id': 'total_products'})
np_array_user_product = user_product.unstack('product_id').values
np_array_user_product = np.nan_to_num(np_array_user_product).astype(int)

In [0]:
np_array_user_product

In [0]:
# Observations: If I remove total products > 500, RMSE score increases from 17 to 34!
